In [58]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australia_fire_db
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires

In [3]:
byStateurl = 'https://en.wikipedia.org/wiki/List_of_fires_and_impacts_of_the_2019-20_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'

In [4]:
historical_response = requests.get(historicalurl)

In [5]:
h_soup = BeautifulSoup(historical_response.text, 'html.parser')

In [6]:
h_table = h_soup.find('table', class_="wikitable")
h_table_headers = h_soup.find_all("tr")[0:2]

In [7]:
# table_headers = table_head.find('tr')
h_headers = []
for table_header in h_table_headers:
    try:
        value = table_header.text
#         value = value.split("\n")
        h_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(h_headers)
print(len(h_headers))

['\nDate\n\nName or description\n\nState(s) /territories\n\nArea burned (approx.)\n\nFatalities\n\nProperties damaged\n\nNotes\n', '\nha\n\nacres\n\nHomes(destroyed)\n\nOther buildings\n\nOther damage\n']
2


In [8]:
h_headers = ['Date', 'Name', 'State(s)/territories', 'AreaBurned(ha)', 'AreaBurned(acres)', 'Fatalities', 'PropertiesDamaged(HomesDestroyed)']

In [9]:
h_table = h_soup.find('table', class_="wikitable")
h_table_body = h_table.find("tbody")
h_table_row = h_table_body.find_all('tr')[2:]

In [10]:
h_data = []

for row in h_table_row:
    
    datarow = []
    
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    h_data.append(datarow)
        
# print(h_data)

[['6 February 1851', 'Black Thursday bushfires', 'Victoria', '5,000,000', '12,000,000', 'approx. 12', 'Nil', 'Nil', '1 million sheepthousands of cattle', '[7][8]'], ['1 February 1898', 'Red Tuesday bushfires', 'Victoria', '260,000', '640,000', '12', 'Nil', '2,000', '', '[8][9][10]'], ['February – March 1926', '1926 bushfires', 'Victoria', '390,000', '960,000', '60', '1,000', 'Nil', '', '[11]'], ['December 1938 – January 1939', 'Black Friday bushfires', 'Victoria', '2,000,000', '4,900,000', '71', '3,700', 'Nil', '', '[5]'], ['14 January – 14 February 1944', '1944 Victorian bushfires', 'Victoria', '1,000,000', '2,500,000', '15–20', 'approx. 500', 'Nil', '', '[8]'], ['18 November 1944', '1944 Blue Mountains bushfire', 'New South Wales', '', '', 'Nil', 'approx. 40', 'Nil', '', '[12][13]'], ['November 1951 – January 1952', '1951–52 bushfires', 'Victoria', '4,000,000', '9,900,000', '11', 'Nil', 'Nil', '', '[14]'], ['2 January 1955', 'Black Sunday bushfires', 'South Australia', '39,000–160,00

In [11]:
h_df = pd.DataFrame(h_data)
h_df
# drop columns from 7-9: done
# clean up Nils to 0 ect: done
# convert string numbers to numeric: doing
# extract year into another column (if two years, only grab the first year): do

,0,1,2,3,4,5,6,7,8,9
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil,Nil,1 million sheepthousands of cattle,[7][8]
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil,"2,000",,[8][9][10]
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000",Nil,,[11]
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700",Nil,,[5]
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500,Nil,,[8]
...,...,...,...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91,Nil,,[66][67]
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181,Yarloop Workshops,Thousands of hectares of Lane Poole ReservePro...,[68]
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35,Nil,,[69]
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19,57,LivestockVineyards,[70][71]


In [12]:
h_drop_rows_df = h_df[[0,1,2,3,4,5,6]]
h_drop_rows_df

,0,1,2,3,4,5,6
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19


In [13]:
h_drop_rows_df.columns = h_headers
h_drop_rows_df

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19


In [106]:
h_nil_df = h_drop_rows_df.replace('Nil', '0')
h_nil_df.head(10)

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,0
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,0
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
5,18 November 1944,1944 Blue Mountains bushfire,New South Wales,,,0,approx. 40
6,November 1951 – January 1952,1951–52 bushfires,Victoria,"4,000,000","9,900,000",11,0
7,2 January 1955,Black Sunday bushfires,South Australia,"39,000–160,000","96,000–395,000",2,40[b]
8,30 November 1957,"1957 Grose Valley bushfire, Blue Mountains",New South Wales,,,4,0
9,2 December 1957,"1957 Leura bushfire, Blue Mountains",New South Wales,,,0,170[c]


In [135]:
# transform columns 4-6 into numeric
# for range values need to get average
# for drop rows with empty values
# remove anything that used to be links, classified as [letter]
# remove words like approx.

h_tonumeric_df = h_nil_df.copy()
h_tonumeric_df['AreaBurned(ha)'] = h_nil_df['AreaBurned(ha)'].str.replace(',','')
h_tonumeric_df['AreaBurned(acres)'] = h_nil_df['AreaBurned(acres)'].str.replace(',','')
h_tonumeric_df['PropertiesDamaged(HomesDestroyed)'] = h_nil_df['PropertiesDamaged(HomesDestroyed)'].str.replace(',','')

columnstoedit = ['AreaBurned(ha)','AreaBurned(acres)','Fatalities','PropertiesDamaged(HomesDestroyed)']

for i in range(len(h_tonumeric_df['Date'])):
    for column in columnstoedit:
        
        if ((h_tonumeric_df[column][i] == '') and (h_tonumeric_df[column][i] != '0')) or (h_tonumeric_df[column][i] == 'unknown') :
            h_tonumeric_df[column][i] = 'NaN'

        if 'approx. ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].replace('approx. ', '')
            
        if 'than ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].split(' ')[-1]

        if len(str(h_tonumeric_df[column][i]).split('–')) == 2:
            splitvalues = str(h_tonumeric_df[column][i]).split('–')
            h_tonumeric_df[column][i] = np.mean([int(splitvalues[0]), int(splitvalues[1])])

        if len(str(h_tonumeric_df[column][i]).split('[')) == 2:
            h_tonumeric_df[column][i] = str(h_tonumeric_df[column][i]).split('[')[0]
        
# h_tonumeric_df = h_tonumeric_df.dropna(axis=1)

h_tonumeric_df.head(10)

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,5000000,12000000,12,0
1,1 February 1898,Red Tuesday bushfires,Victoria,260000,640000,12,0
2,February – March 1926,1926 bushfires,Victoria,390000,960000,60,1000
3,December 1938 – January 1939,Black Friday bushfires,Victoria,2000000,4900000,71,3700
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,1000000,2500000,17.5,500
5,18 November 1944,1944 Blue Mountains bushfire,New South Wales,NaN,NaN,0,40
6,November 1951 – January 1952,1951–52 bushfires,Victoria,4000000,9900000,11,0
7,2 January 1955,Black Sunday bushfires,South Australia,99500,245500,2,40
8,30 November 1957,"1957 Grose Valley bushfire, Blue Mountains",New South Wales,NaN,NaN,4,0
9,2 December 1957,"1957 Leura bushfire, Blue Mountains",New South Wales,NaN,NaN,0,170


In [136]:
# df[df.name != 'Tina']

h_tonumeric_df = h_tonumeric_df[h_tonumeric_df['AreaBurned(ha)'] != 'NaN']
h_tonumeric_df = h_tonumeric_df[h_tonumeric_df['AreaBurned(ha)'] != 'NaN']
h_tonumeric_df = h_tonumeric_df.astype(
    {
        'AreaBurned(ha)':'int',
        'AreaBurned(acres)': 'int',
        'Fatalities':'int',
        'PropertiesDamaged(HomesDestroyed)':'int'
    })
h_tonumeric_df.dtypes

ValueError: invalid literal for int() with base 10: 'NaN'

In [ ]:
# fires_modis_dict = modis_df.to_dict('range')

h_dict = h_df.to_dict('range')
h_dict